## Imputation/Engineering revised

This is a rewrite of a previous version. Soft impute is a matrix completion algorithm for missing values based off soft thresholding. Previously, imputations were performed one col at a time - totally unnecessary. Additionally, the normalization using StandardScalar() (required ahead of imputation) preserves nulls. Therefore, the custom script was also unnecessary.   

This version abstracts away most of the engineering/cleaning into classes that are (hopefully) much easier to follow and continue to evolve. Additionally, wrappers are placed around SoftImpute and StandardScalar to allow sklearn pipeline use.

In [1]:

import sys
path = '../code_datalab/'
for folder in ['quickpipeline', 'features', 'datalab', 'pd_feature_union',
              'libraries']:
    sys.path.append(path + folder)

from datalab import *
from quickpipe_mod import *
from features import *
from pandas_feature_union import *

# THE PATHS ARE ARE LOCAL SO THE FOLDER CAN WORK WITHIN ITSELF. IF CERTAIN FOLDERS ARE MOVED, THE 
# IMPORTS WILL STOP WORKING
from libraries import *
from preproc import*
%matplotlib inline

Using TensorFlow backend.


### 1. Import Data

In [20]:
# IMPORT DATA
csv = pd.read_csv('../data/data_sets/APR.csv')
data = csv[( (csv.BOG_FTPT == 'Full Time') & (csv.STU_TYPE.isin(['B', 'E'])) )]
# THE MASTER DICT IS DEFINED IN THE DATALAB CODE FILE
data['cohort'] = data['STU_ID'].map(master_dict)
data = data[data['STU_COLLEGE'] == 'A']
data.reset_index(drop=True, inplace=True)

# LABEL
y = pd.DataFrame(data.APR)
y = pd.DataFrame(y['APR'].apply(lambda x: 1 if x == 0 else 0))
y.rename(columns = {'APR': 'churn'}, inplace=True)
data.drop(['APR', 'CHM2045LTaken', 'BSC2010LTaken'], axis=1, inplace=True)

In [3]:
# DOES NOT INCLUDE GPA FOR THE FIRST SEMESTER
feats = ['STU_AGE', 'STU_DIVISION', 'STU_ENTER_SUMMER', 'AP_IND', 'GENDER_CODE', 
            'FEE_RESIDENCY', 'STU_HONORS', 'STU_ONLINE', 
            'STU_TERM_LOAD', 'HIGHDEG_HELD', 'FIRST_GEN', 'AP_CREDITS', 'TESTSCORE', 
         'BESTTEST', 'FL', 'cohort', 
            'FIN_AWARD', 'BANNER_PROGRAM', 'COUNTY', 'STU_STATE', 'STU_NATION', 
       'ADMISSION_TERM', 'GPA_HIGHSCHOOL', 'STU_DEPARTMENT', 
       'RESIDENCE', 'FININC', 'FINAID_NEED', 'FINCONTRIB', 'ETHNICITY_CODE',
        'STU_CLASSIFICATION']

### 2. Define Data Frames

In [4]:
# USEFUL CLASS. BREAKS AWAY AND REDUCES TO RELEVANT COURSES
transformer = CourseSeparater(data)
# RETURNS 3 DFRAMES FOR CORE=TRUE; 2 IF CORE=FALSE 
df_non, df_course, df_core = transformer.course_df(core=True)
df_non = df_non[feats]

In [5]:
# WENT WITH THE COMMON CLASSES
df = pd.concat([df_non, df_core], axis=1)
# SEE SCRIPT FOR TRANSFORMATION/CLEANING - NOT A LOT
FeatureMaps(df).run_all()

In [6]:
df.head()

,STU_AGE,STU_DIVISION,STU_ENTER_SUMMER,AP_IND,GENDER_CODE,FEE_RESIDENCY,STU_HONORS,STU_ONLINE,STU_TERM_LOAD,HIGHDEG_HELD,FIRST_GEN,AP_CREDITS,TESTSCORE,BESTTEST,FL,cohort,FIN_AWARD,BANNER_PROGRAM,COUNTY,STU_STATE,GPA_HIGHSCHOOL,STU_DEPARTMENT,FININC,FINAID_NEED,FINCONTRIB,MAC1105Taken,ZOO1010Taken,ZOO1010LTaken,ENC1101Taken,SLS1109Taken,AMH2020Taken,MAT1033Taken,CHM2045Taken,PSY2012Taken,MAC1140Taken,MUH2930Taken,MAC2311Taken,BOT2010Taken,BOT2010LTaken,BSC2010Taken,BSC2844Taken,ETHNICITY_CODE_cat,STU_CLASSIFICATION_cat,Commuter,US_res,Summer_adm
0,22,L,True,False,M,F,False,No Online Course,13,H,False,0,0.375000,SAT,True,2008,250.00,U-MBY-BS-00,SANT,FL,2.9,BY,NaN,NaN,NaN,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,cat_7,cat_1,True,True,True
1,18,L,False,False,M,F,False,No Online Course,16,H,False,0,0.611111,ACT,True,2008,2683.00,U-BLY-BS-01,ESCA,FL,3.9,BY,50154.0,4976.0,9297.0,True,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,cat_4,cat_1,True,True,False
2,18,L,False,True,F,F,True,No Online Course,13,H,False,3,0.694444,ACT,True,2008,3149.00,U-BLY-BS-00,ESCA,FL,NaN,BY,57336.0,966.0,4567.0,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,cat_7,cat_1,True,True,False
3,18,L,False,True,M,F,False,No Online Course,13,H,False,3,0.722222,ACT,True,2008,1213.00,U-MBY-BS-00,OKAL,FL,3.9,BY,NaN,NaN,NaN,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,cat_7,cat_1,True,True,False
4,18,L,False,False,M,F,False,No Online Course,13,H,False,0,0.777778,ACT,True,2008,6337.42,U-MAT-BS-00,SANT,FL,NaN,MAT,NaN,3402.0,799.0,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,cat_2,cat_1,True,True,False


### 3. Pipeline Walkthrough

In [7]:
# WALKTHROUGH FOR CATEGORICAL PIPE
cat = TypeSelector(np.object)
df_cat = cat.fit_transform(df)
pipe = FreqFeatures(min_=10)
df_pipe = pipe.fit_transform(df_cat)
qp = QuickPipeline_mod()
qp.fit_transform(df_pipe).head(3)

,STU_DIVISION,GENDER_CODE,BESTTEST,BANNER_PROGRAM_freq,COUNTY_freq,STU_STATE_freq,STU_DEPARTMENT_freq,FEE_RESIDENCY_A,FEE_RESIDENCY_E,FEE_RESIDENCY_F,FEE_RESIDENCY_N,FEE_RESIDENCY_R,FEE_RESIDENCY_T,STU_ONLINE_No Online Course,STU_ONLINE_Online Only,STU_ONLINE_Partial Online,HIGHDEG_HELD_A,HIGHDEG_HELD_C,HIGHDEG_HELD_G,HIGHDEG_HELD_H,cohort_2008,cohort_2009,cohort_2010,cohort_2011,cohort_2012,cohort_2013,cohort_2014,cohort_2015,cohort_2016,cohort_2017,ETHNICITY_CODE_cat_cat_1,ETHNICITY_CODE_cat_cat_2,ETHNICITY_CODE_cat_cat_3,ETHNICITY_CODE_cat_cat_4,ETHNICITY_CODE_cat_cat_5,ETHNICITY_CODE_cat_cat_6,ETHNICITY_CODE_cat_cat_7,ETHNICITY_CODE_cat_cat_8,ETHNICITY_CODE_cat_cat_9,STU_CLASSIFICATION_cat_cat_1,STU_CLASSIFICATION_cat_cat_2,STU_CLASSIFICATION_cat_cat_3,STU_CLASSIFICATION_cat_cat_4,BANNER_PROGRAM_reduce_U-BLY-BS-00,BANNER_PROGRAM_reduce_U-BLY-BS-01,BANNER_PROGRAM_reduce_U-BMD-BS-00,BANNER_PROGRAM_reduce_U-CHM-BA-00,BANNER_PROGRAM_reduce_U-CHM-BA-01,BANNER_PROGRAM_reduce_U-CHM-BS-00,BANNER_PROGRAM_reduce_U-CHM-BS-01,BANNER_PROGRAM_reduce_U-CPE-BSCE00,BANNER_PROGRAM_reduce_U-CPE-BSCE0P,BANNER_PROGRAM_reduce_U-CPS-BS-00,BANNER_PROGRAM_reduce_U-CPS-BS-01,BANNER_PROGRAM_reduce_U-CPS-BS-03,BANNER_PROGRAM_reduce_U-CPS-BS-05,BANNER_PROGRAM_reduce_U-CPS-BS-06,BANNER_PROGRAM_reduce_U-CPS-BS-07,BANNER_PROGRAM_reduce_U-CPS-BS-08,BANNER_PROGRAM_reduce_U-EEG-BSEE00,BANNER_PROGRAM_reduce_U-EEG-BSEE0P,BANNER_PROGRAM_reduce_U-ENV-BS-01,BANNER_PROGRAM_reduce_U-ENV-BS-02,BANNER_PROGRAM_reduce_U-INF-BS-03,BANNER_PROGRAM_reduce_U-MAT-BS-00,BANNER_PROGRAM_reduce_U-MBY-BS-00,BANNER_PROGRAM_reduce_U-MEC-BSME00,BANNER_PROGRAM_reduce_U-MEC-BSME0P,BANNER_PROGRAM_reduce_U-PHY-BS-00,BANNER_PROGRAM_reduce_U-PHY-BS-01,BANNER_PROGRAM_reduce_less_common,BANNER_PROGRAM_reduce_rare,COUNTY_reduce_ALAC,COUNTY_reduce_BAY,COUNTY_reduce_BREV,COUNTY_reduce_BROW,COUNTY_reduce_CITR,COUNTY_reduce_CLAY,COUNTY_reduce_DADE,COUNTY_reduce_DUVA,COUNTY_reduce_ESCA,COUNTY_reduce_HERN,COUNTY_reduce_HILL,COUNTY_reduce_JACK,COUNTY_reduce_LAKE,COUNTY_reduce_LEE,COUNTY_reduce_LEON,COUNTY_reduce_MANA,COUNTY_reduce_MARI,COUNTY_reduce_NFLA,COUNTY_reduce_NUSA,COUNTY_reduce_OKAL,COUNTY_reduce_ORAN,COUNTY_reduce_OSCE,COUNTY_reduce_PALM,COUNTY_reduce_PASC,COUNTY_reduce_PINE,COUNTY_reduce_POLK,COUNTY_reduce_SANT,COUNTY_reduce_SEMI,COUNTY_reduce_ST J,COUNTY_reduce_VOLU,COUNTY_reduce_WALT,COUNTY_reduce_less_common,COUNTY_reduce_rare,STU_STATE_reduce_AL,STU_STATE_reduce_FL,STU_STATE_reduce_GA,STU_STATE_reduce_MD,STU_STATE_reduce_TX,STU_STATE_reduce_XX,STU_STATE_reduce_less_common,STU_STATE_reduce_rare,STU_DEPARTMENT_reduce_BY,STU_DEPARTMENT_reduce_CHM,STU_DEPARTMENT_reduce_ECP,STU_DEPARTMENT_reduce_EVR,STU_DEPARTMENT_reduce_MAT,STU_DEPARTMENT_reduce_MEC,STU_DEPARTMENT_reduce_PHY,STU_DEPARTMENT_reduce_SS,STU_DEPARTMENT_reduce_rare
0,0,1,1,0.742400,0.676363,0.3467,1.005665,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,1.582764,1.310095,0.3467,1.005665,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0.025194,1.310095,0.3467,1.005665,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [10]:
# WALKTHROUGH FOR BOOL PIPE
# UNFORTUNATELY, QUICKPIPE IS HAVING ISSUES WITH BOOLS. WE'LL 
# NEED TO SPECIFY THE COLNAMES IN THE HYPERPARAMETERS
bools = TypeSelector(np.bool)
df_bool = bools.fit_transform(df)
bool_names = df_bool.columns
qp = QuickPipeline_mod(categorical_features=df_bool.columns)

qp.fit_transform(df_bool).head(3)

,STU_ENTER_SUMMER,AP_IND,STU_HONORS,FIRST_GEN,FL,MAC1105Taken,ZOO1010Taken,ZOO1010LTaken,ENC1101Taken,SLS1109Taken,AMH2020Taken,MAT1033Taken,CHM2045Taken,PSY2012Taken,MAC1140Taken,MUH2930Taken,MAC2311Taken,BOT2010Taken,BOT2010LTaken,BSC2010Taken,BSC2844Taken,Commuter,US_res,Summer_adm
0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1
1,0,0,0,0,1,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0
2,0,1,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0


In [9]:
# WALKTHROUGH FOR NUM PIPE WITH IMPUTATION
num = TypeSelector(np.number)
df_num = num.fit_transform(df)
num = StandardScalerDf()
df_num = num.fit_transform(df_num)
# INITIALLY WORRIED THAT THIS DID NOT PRESERVE NULLS - IT DOES
num = SoftImputeDf()
num.fit_transform(df_num).head(3)

,STU_AGE,STU_TERM_LOAD,AP_CREDITS,TESTSCORE,FIN_AWARD,GPA_HIGHSCHOOL,FININC,FINAID_NEED,FINCONTRIB
0,2.451414,-0.367031,-0.433675,-1.943986,-1.002111,-0.974976,-0.013465,0.029399,0.020895
1,-0.365499,2.054997,-0.433675,-0.180338,-0.213867,0.642642,-0.380213,-0.361470,-0.173695
2,-0.365499,-0.367031,0.109812,0.442126,-0.062892,0.080354,-0.276680,-0.753657,-0.372330


### 4. Preprocessing Pipeline

In [11]:
# THESE CAN ALL BE RUN SEPARATELY IF NEEDED
transformer_list=[
        ("binned_features", make_pipeline(
                        TypeSelector(np.number),
                        StandardScalerDf(),
                        SoftImputeDf(),
                        SelectFeatures(),
                        KBins()
        )),
        ("numeric_features", make_pipeline(
                            TypeSelector(np.number),
                            StandardScalerDf(),
                            SoftImputeDf()
        )),
        ("categorical_features", make_pipeline(
                             TypeSelector(np.object),
                             FreqFeatures(min_=10),  
                             QuickPipeline_mod() 
        )),
        ("boolean_features", make_pipeline(
                         TypeSelector(np.bool_),
                         QuickPipeline_mod(categorical_features=bool_names) 
        ))  ]

In [12]:
preprocess_pipeline = PandasFeatureUnion(transformer_list)

In [13]:
X = preprocess_pipeline.fit_transform(df)

In [18]:
# PREPROCESSED DATA WITH IMPUTATION
final = pd.concat([X, y], axis=1)

### 5. Final output

In [21]:
final.shape

(3867, 162)